In [1]:
'''
dictionary = something belongs to something else, is kwargs, or has certain order
list = collection of several parallel operations

blocks -> first level keys (THESE HAVE SET ORDER)
Below this ^^^ -> The order you provide the code will matter
    - dict = matters
    - list = does not matter

'''
import sys
import os
#print(os.getcwd())
sys.path.append("../")
from sklearn.preprocessing import *
from sklearn.model_selection import *
from scipy.stats import *
import pandas as pd
from sklearn.metrics import *
from ml_airflow.static.preprocessing import *
from ml_airflow.static.splitting import *
from ml_airflow.static.feature_engineering import *
from ml_airflow.static.eda import *
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from ml_airflow.dag.layers import DagLayer

kwargs = \
{   
     'modeling': { 
                    'models': DagLayer({'LOG': {LogisticRegression: None}, 
                                       'GNB': {GaussianNB: None},
                                       'RF':{RandomForestClassifier: {'n_estimators': 10}},
                                       'MLP': {MLPClassifier: {'activation': 'tanh'}},
                                       'SVM': {SVC: {'kernel': 'linear'}},
                                       'KNN': {KNeighborsClassifier: {'n_neighbors':3 }}}),              
                 },
 
     'data_sources': [DagLayer({'https://raw.githubusercontent.com/SamShowalter/WMP_training/01_EDA/data/IBM_Employee_Attrition.csv': {pd.read_csv:None}})],

#Expects flat dataset
    'splitting': [
                        DagLayer({'split': {auto_train_test_split  : {'target': "Attrition",
                                                                        'test_ratio': 0.2,
                                                                         'random_state': 42}}}),
                 ],
 
     
     #Expects flat dataset
        'preprocessing': { 
                        
                        'cleansing': [   
                                              DagLayer({'winsorize': {mla_winsorize: {'limits' : [0.05, 0.05],
                                                                                      'col_names': ['MonthlyRate',
                                                                                                    'NumCompaniesWorked',
                                                                                                   'PercentSalaryHike']}}}),
                                              DagLayer({'median_impute': {mla_impute : {'method': "median"}}})
                                     ]
                        
                     },
    #Expects flat dataset
    'feature_engineering': {
                            'transformations': DagLayer({
                                    ('Department', 'EducationField', 'JobRole', "MaritalStatus"): {pd.get_dummies: None},
                                    'BusinessTravel': {create_ordinal_df: {'ordinal_dict':{"Non-Travel" :0, 
                                                                                            "Travel_Rarely":1, 
                                                                                            "Travel_Frequently": 2},
                                                                          'tag': None}},
                                    ('Attrition', 'Gender', 'OverTime') : {create_boolean_df: 
                                                                           {'boolean_names_and_values':
                                                                            {"Attrition": ["Yes", "No"],
                                                                            "Gender": ["Male", "Female"],
                                                                            "OverTime": ["Yes", "No"]}, 
                                                                            'tag' : None}},
                                    ('Age', 'DistanceFromHome', "EnvironmentSatisfaction",
                                    'JobInvolvement', 'JobSatisfaction', "MonthlyIncome",
                                    'NumCompaniesWorked', 'PercentSalaryHike', "PerformanceRating",
                                    'RelationshipSatisfaction', 'TotalWorkingYears',
                                    'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
                                    'YearsInCurrentRole', 'YearsSinceLastPromotion',
                                    'YearsWithCurrManager') : {normalize_values: {'tag': None}},
                                    'MonthlyRate': {mla_linear_transformation: {'method': boxcox},
                                                    normalize_values: {'tag': None}},
                               })
                            
                           },
 
     #Expects predictions
     'evaluation':{'metrics' : DagLayer({'acc': {accuracy_score: None}, 
                                        'f1': {f1_score: None}, 
                                        'precision': {precision_score: None},
                                        'conf_matrix': {confusion_matrix: None}, 
                                        'class_report': {classification_report: None},
                                        'recall': {recall_score :None}})
                  }
   

}

In [53]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/SamShowalter/WMP_training/01_EDA/data/IBM_Employee_Attrition.csv')

In [54]:
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   1
Department                  0
DistanceFromHome            2
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              1
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              1
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               2
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [55]:
t = df.mean()
tdf = df.loc[:,["DailyRate", "JobInvolvement"]]

In [65]:
isinstance(df, tuple)

False

In [63]:
def mla_impute(data, method = "median", prefit = None):

	fill_na_vals = None
	if prefit:
		fill_na_vals = prefit['fill_na_vals']

	else:
		if method == 'median':
			fill_na_vals = data.median()
		elif method == 'mean':
			fill_na_vals = data.mean()

	data = data.fillna(fill_na_vals)

	if prefit:
		return data
	else:
		return data, {'fill_na_vals': fill_na_vals}


In [64]:
mla_impute(df)

(      Age Attrition     BusinessTravel  DailyRate              Department  \
 0      41       Yes      Travel_Rarely     1102.0                   Sales   
 1      49        No  Travel_Frequently      279.0  Research & Development   
 2      37       Yes      Travel_Rarely     1373.0  Research & Development   
 3      33        No  Travel_Frequently     1392.0  Research & Development   
 4      27        No      Travel_Rarely      591.0  Research & Development   
 ...   ...       ...                ...        ...                     ...   
 1465   36        No  Travel_Frequently      884.0  Research & Development   
 1466   39        No      Travel_Rarely      613.0  Research & Development   
 1467   27        No      Travel_Rarely      155.0  Research & Development   
 1468   49        No  Travel_Frequently     1023.0                   Sales   
 1469   34        No      Travel_Rarely      628.0  Research & Development   
 
       DistanceFromHome  Education EducationField  EmployeeCou

In [4]:
import pandas as pd
import os
import sys
sys.path.append("../ml_airflow/static")
sys.path.append("../ml_airflow/dag")

In [5]:
from generator import DagGenerator
from datetime import datetime



In [6]:
dag_args = { 
                'dag_name': "ML_Airflow_test_creation_full",
                'dag':      {
                                'owner': 'Sam Showalter',
                                'email': ['sshowalter@wmp.com'],
                                'op_args':{},
                                'op_kwargs': {},
                                'params': {}
                            },
                'config' : kwargs
}

In [7]:
dg = DagGenerator(dag_args)
dg.generate_file()


Displaying Ordered Dag Layers with Tags:

0 ['data_sources'] ds 0
1 ['preprocessing', 'cleansing'] p_cleansing_l0 0
2 ['preprocessing', 'cleansing'] p_cleansing_l1 0
3 ['feature_engineering', 'transformations'] fe_transformations 0
4 ['splitting'] s 0
5 ['modeling', 'models'] m_models 0
6 ['evaluation', 'metrics'] e_metrics 1

Generated airflow file saved at: /home/sshowalter/airflow/dags/ML_Airflow_test_creation_full_Sam-Showalter_10-17-2019--16.31.29.py
